# Confflow Demo

## Imports

In [ ]:
from typing import Literal, Optional

from confflow import BaseConfig, Field, config_manager

# Settings

In [ ]:
class CommonSettings(BaseConfig):
    name: str = Field(..., min_length=3, max_length=50, description="The name of the configuration.")
    enabled: bool = Field(default=True, description="Indicates if this configuration is enabled.")
    priority: int = Field(default=1, ge=1, le=10, description="Priority level, must be between 1 and 10.")


class DatabaseConfig(BaseConfig):
    db_url: str = Field(..., pattern=r"^(postgres|mysql|sqlite)://", description="Database connection URL.")
    max_connections: int = Field(default=10, ge=1, le=100, description="Max number of connections.")
    timeout: Optional[int] = Field(default=30, ge=10, le=300, description="Timeout in seconds.")


class APIConfig(BaseConfig):
    endpoint: str = Field(..., pattern=r"^https?://", description="API endpoint URL.")
    auth_token: Optional[str] = Field(None, description="Optional authentication token.")
    retries: int = Field(default=3, ge=0, le=10, description="Number of retries in case of failure.")


class FeatureFlags(BaseConfig):
    experimental_feature: bool = Field(default=False, description="Toggle for experimental feature.")
    legacy_mode: bool = Field(default=False, description="Enable legacy mode for backward compatibility.")


# Mutually Exclusive Models
class FileStorageConfig(BaseConfig):
    storage_path: str = Field(..., description="Path to the storage directory.")
    max_size_mb: int = Field(default=100, ge=10, le=1024, description="Maximum storage size in MB.")
    backup_enabled: bool = Field(default=True, description="Enable backup for stored files.")


class CloudStorageConfig(BaseConfig):
    provider: Literal['aws', 'gcp', 'azure'] = Field(..., description="Cloud storage provider.")
    bucket_name: str = Field(..., min_length=3, description="Name of the cloud storage bucket.")
    region: Optional[str] = Field(None, description="Optional region of the cloud storage bucket.")

In [ ]:
config_manager.register_schemas(
    CommonSettings,
    FeatureFlags,
    CloudStorageConfig,
    DatabaseConfig
)

In [ ]:
config_manager.set_mutual_exclusive_groups([CloudStorageConfig, DatabaseConfig])

In [ ]:
config_manager.create_template("template_config.yml")

In [ ]:
config_manager.load_yaml("config.yml")

In [ ]:
config_manager["CommonSettings"]

In [ ]:
config_manager["CommonSettings"]["enabled"]